In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

# Global variable to store the currently logged-in user ID
logged_in_user_id = None

# Function to handle login
def login():
    global logged_in_user_id

    username = username_entry.get()
    password = password_entry.get()

    # Establish a connection to the SQLite database
    conn = sqlite3.connect('store.db')
    cursor = conn.cursor()

    # Check username and password against the Users table in the database
    cursor.execute("SELECT * FROM Users WHERE username = ? AND password = ?", (username, password))
    user = cursor.fetchone()

    if user is not None:
        logged_in_user_id = user[0]  # Store the logged-in user ID
        messagebox.showinfo("Login", "Login Successful")
        # Enable product listing, product update, order placement, sell item, and logout functionality
        list_product_btn.config(state=tk.NORMAL)
        make_order_btn.config(state=tk.NORMAL)
        sell_item_btn.config(state=tk.NORMAL)
        logout_btn.config(state=tk.NORMAL)
        # Reset the order list
        selected_products.clear()
    else:
        messagebox.showerror("Login Error", "Invalid username or password")
        # Disable product listing, product update, order placement, sell item, and logout functionality
        list_product_btn.config(state=tk.DISABLED)
        make_order_btn.config(state=tk.DISABLED)
        sell_item_btn.config(state=tk.DISABLED)
        logout_btn.config(state=tk.DISABLED)

    # Close the database connection
    conn.close()

# Function to handle logout
def logout():
    global logged_in_user_id
    logged_in_user_id = None
    messagebox.showinfo("Logout", "Logout Successful")
    # Disable product listing, product update, order placement, sell item, and logout functionality
    list_product_btn.config(state=tk.DISABLED)
    make_order_btn.config(state=tk.DISABLED)
    sell_item_btn.config(state=tk.DISABLED)
    logout_btn.config(state=tk.DISABLED)
    # Reset username and password fields
    username_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)

# Function to handle product listing
def list_product():
    # Check if the user is logged in
    if list_product_btn['state'] == tk.DISABLED:
        messagebox.showerror("Access Denied", "Please login to access this functionality")
        return

    # Create a new window for the product list
    product_list_window = tk.Toplevel()
    product_list_window.title("Product List")

    # Create a frame to hold the listbox and scrollbar
    list_frame = tk.Frame(product_list_window)
    list_frame.pack(padx=10, pady=10)

    # Create a scrollbar
    scrollbar = tk.Scrollbar(list_frame)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    # Create a listbox to display the product list
    product_listbox = tk.Listbox(list_frame, width=120, height=50, yscrollcommand=scrollbar.set)
    product_listbox.pack(side=tk.LEFT, fill=tk.BOTH)

    # Configure the scrollbar
    scrollbar.config(command=product_listbox.yview)

    # Function to handle product selection
    def select_product():
        # Get the selected product
        selected_index = product_listbox.curselection()
        if selected_index:
            selected_product = products[selected_index[0]]
            selected_products.append(selected_product[0])
            messagebox.showinfo("Product Selection", "Product added to the order")
        else:
            messagebox.showerror("Product Selection Error", "Please select a product")

    # Create a button to select a product
    select_btn = tk.Button(product_list_window, text="Select", command=select_product)
    select_btn.pack(pady=10)

    # Establish a connection to the SQLite database
    conn = sqlite3.connect('store.db')
    cursor = conn.cursor()

    # Retrieve the list of products from the Products table
    cursor.execute("SELECT * FROM Products")
    products = cursor.fetchall()

    # Add the products to the listbox
    for product in products:
        product_listbox.insert(tk.END, f"Product ID: {product[0]} | Product Name: {product[2]} | Description: {product[3]} | Quantity: {product[5]}")

    # Close the database connection
    conn.close()

# Function to handle order placement
def make_order():
    # Check if the user is logged in
    if make_order_btn['state'] == tk.DISABLED:
        messagebox.showerror("Access Denied", "Please login to access this functionality")
        return

    # Check if any products are selected
    if not selected_products:
        messagebox.showerror("Order Placement Error", "No products selected")
        return

    # Create a new window for the order placement
    order_window = tk.Toplevel()
    order_window.title("Make Order")

    # Implement order placement logic here
    messagebox.showinfo("Make Order", "Order Placed Successfully")

# Function to handle selling a new item
def sell_item():
    # Check if the user is logged in
    if sell_item_btn['state'] == tk.DISABLED:
        messagebox.showerror("Access Denied", "Please login to access this functionality")
        return

    # Create a new window for selling an item
    sell_item_window = tk.Toplevel()
    sell_item_window.title("Sell Item")

    # Create a frame for sell item form
    sell_item_frame = tk.Frame(sell_item_window)
    sell_item_frame.pack(padx=10, pady=10)

    # Create labels and entry fields for product details
    product_name_label = tk.Label(sell_item_frame, text="Product Name:")
    product_name_label.grid(row=0, column=0, padx=5, pady=5)
    product_name_entry = tk.Entry(sell_item_frame)
    product_name_entry.grid(row=0, column=1, padx=5, pady=5)

    description_label = tk.Label(sell_item_frame, text="Description:")
    description_label.grid(row=1, column=0, padx=5, pady=5)
    description_entry = tk.Entry(sell_item_frame)
    description_entry.grid(row=1, column=1, padx=5, pady=5)

    price_label = tk.Label(sell_item_frame, text="Price:")
    price_label.grid(row=2, column=0, padx=5, pady=5)
    price_entry = tk.Entry(sell_item_frame)
    price_entry.grid(row=2, column=1, padx=5, pady=5)

    quantity_label = tk.Label(sell_item_frame, text="Quantity:")
    quantity_label.grid(row=3, column=0, padx=5, pady=5)
    quantity_entry = tk.Entry(sell_item_frame)
    quantity_entry.grid(row=3, column=1, padx=5, pady=5)

    # Function to handle product listing
    def sell_product():
        product_name = product_name_entry.get()
        description = description_entry.get()
        price = float(price_entry.get())
        quantity = int(quantity_entry.get())

        # Check for valid inputs
        if not product_name or not description or price <= 0 or quantity <= 0:
            messagebox.showerror("Sell Item Error", "Please enter valid product details")
            return

        # Establish a connection to the SQLite database
        conn = sqlite3.connect('store.db')
        cursor = conn.cursor()

        # Insert the new product into the Products table
        cursor.execute("INSERT INTO Products (UserID, Name, Description, Price, Quantity) VALUES (?, ?, ?, ?, ?)",
                       (logged_in_user_id, product_name, description, price, quantity))
        conn.commit()

        messagebox.showinfo("Sell Item", "Product listed successfully and updated the product list")
        sell_item_window.destroy()  # Close the sell item window

        # Update the product list
        list_product()

        # Close the database connection
        conn.close()

    # Create a button to sell the product
    sell_btn = tk.Button(sell_item_frame, text="Sell", command=sell_product)
    sell_btn.grid(row=4, columnspan=2, padx=5, pady=10)

# Create the main window
window = tk.Tk()
window.title("Online Shopping System")

# Login Frame
login_frame = tk.Frame(window)
login_frame.pack(padx=10, pady=10)

username_label = tk.Label(login_frame, text="Username:")
username_label.grid(row=0, column=0, padx=5, pady=5)
username_entry = tk.Entry(login_frame)
username_entry.grid(row=0, column=1, padx=5, pady=5)

password_label = tk.Label(login_frame, text="Password:")
password_label.grid(row=1, column=0, padx=5, pady=5)
password_entry = tk.Entry(login_frame, show="*")
password_entry.grid(row=1, column=1, padx=5, pady=5)

login_btn = tk.Button(login_frame, text="Login", command=login)
login_btn.grid(row=2, columnspan=2, padx=5, pady=5)

# Product Frame
product_frame = tk.Frame(window)
product_frame.pack(padx=10, pady=10)

list_product_btn = tk.Button(product_frame, text="List Product", command=list_product, state=tk.DISABLED)
list_product_btn.grid(row=0, padx=5, pady=5)

make_order_btn = tk.Button(product_frame, text="Make Order", command=make_order, state=tk.DISABLED)
make_order_btn.grid(row=1, padx=5, pady=5)

sell_item_btn = tk.Button(product_frame, text="Sell Item", command=sell_item, state=tk.DISABLED)
sell_item_btn.grid(row=2, padx=5, pady=5)

logout_btn = tk.Button(product_frame, text="Logout", command=logout, state=tk.DISABLED)
logout_btn.grid(row=3, padx=5, pady=5)

# Run the main event loop
window.mainloop()
